In [1]:
import torch
import torch.autograd as autograd
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import string
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

%load_ext autoreload
%autoreload 2

from rnn import RNN, PrepareData

In [2]:
dataset = PrepareData('input.txt', batch_size=30, val_ratio=0.2)
train_data, val_data = dataset.dataset()

In [3]:
print(len(train_data), len(val_data))
print(train_data[0], val_data[0])

13073 3268
L:1/4
K:F
F | fff | f2e | ddd  n-barndance-11
M:C|
L:1/8
K:D



In [30]:
#input_dim = 7
hidden_dim = 7

all_letters = string.printable
n_letters = len(all_letters)

model = RNN(n_letters, hidden_dim, n_letters)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)

In [31]:
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return Variable(tensor.cuda())
        
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1)
    return Variable(torch.LongTensor(letter_indexes).cuda())

In [ ]:
n_epochs = 2
all_losses = []
print_every = 500

for epoch in range(n_epochs):
    train_cnt = 0
    total_loss = 0.
    
    dataset.shuffle_data()
    train_data, val_data = dataset.dataset()
    
    for sentence in train_data:
        model.zero_grad()
        model.hidden = model.init_hidden()
        
        loss = 0.
        inputs = inputTensor(sentence)
        targets = targetTensor(sentence)
        
        for i in range(len(sentence)-1):
            preds = model(inputs[i])
            loss += criterion(preds, targets[i])
            
        loss.backward()
        optimizer.step()
        total_loss += loss
        train_cnt += 1
        
        if train_cnt % print_every == 0:
            total_loss /= print_every
            all_losses.append(total_loss)
            print('[%d, %d], loss: %f' % (epoch, train_cnt, total_loss))
            total_loss = 0

[0, 500], loss: 132.885757
[0, 1000], loss: 131.669846
[0, 1500], loss: 130.179596
[0, 2000], loss: 127.294518
[0, 2500], loss: 123.547668
[0, 3000], loss: 119.999069
[0, 3500], loss: 116.333557
[0, 4000], loss: 113.134216
[0, 4500], loss: 111.528671
[0, 5000], loss: 110.183228
[0, 5500], loss: 109.165802
[0, 6000], loss: 107.766838
[0, 6500], loss: 107.707375
[0, 7000], loss: 106.555031
[0, 7500], loss: 107.369011
[0, 8000], loss: 107.365456
[0, 8500], loss: 106.345200
[0, 9000], loss: 106.037582
[0, 9500], loss: 106.556557
[0, 10000], loss: 106.358597
[0, 10500], loss: 105.435791
[0, 11000], loss: 106.147797
[0, 11500], loss: 104.682114
[0, 12000], loss: 105.323334
[0, 12500], loss: 105.718254
[0, 13000], loss: 105.167580
[1, 500], loss: 104.775650
[1, 1000], loss: 104.621872
[1, 1500], loss: 105.871231


In [ ]:
max_length = 20

test_sen = val_data[0]
inputs = inputTensor(test_sen)
for i in range(len(test_sen)-1):
    output = model(inputs[i])
    _, pred = torch.max(output.data, 1)
    print(all_letters[int(pred)])
print(test_sen)

In [ ]:
plt.figure()
plt.plot(all_losses)